In [6]:
search_for = 111
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.008028984069824219
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 233072741
type: [1, 1, 1, 1, 111] 111
cases of this type: 151807041
10000 23.582111700850604
20000 23.939320096460403
30000 21.103412050717765
40000 46.232530782061964
50000 89.42521958195148
60000 101.47161961695869
70000 79.83382379937912
80000 77.96660146577594
90000 85.13618685559587
100000 78.86174875065934
110000 105.36805080824394
120000 82.63893364070702
130000 87.47359150945564
140000 104.05521673350202
150000 101.50336563284097
160000 117.39669385770848
170000 135.24828912098636
180000 151.56315700433308
190000 205.12702855862221
200000 176.28661495216565
210000 164.39377365926308
220000 199.37828611051137
230000 190.66157157410066
240000 182.9147567292166
250000 151.66493596819052
260000 192.51405325795238
270000 200.38035391014188
280000 178.75562790463792
290000 156.09059895035935
300000 164.6619765004391
310000 164

3050000 153.69176340324898
3060000 155.11744500743472
3070000 165.28059153578596
3080000 168.9697624168182
3090000 144.20917193844406
3100000 147.86079830244014
3110000 149.20515883132194
3120000 152.3624617761421
3130000 136.80191811109512
3140000 150.96685846897282
3150000 153.23327089713234
3160000 147.81848628308092
3170000 143.7749818995046
3180000 142.4136690993876
3190000 148.4214477673927
3200000 139.38851978492107
3210000 139.60924075306815
3220000 151.98684799842843
3230000 138.81939537679628
3240000 155.8600363275484
3250000 143.67341219836453
3260000 142.72643108752567
3270000 147.21813734906286
3280000 144.17669431822324
3290000 148.85840673806328
3300000 157.33855539959356
3310000 135.40774336244652
3320000 149.93168378072713
3330000 148.26765316124383
3340000 141.2748264123892
3350000 159.66473079543607
3360000 137.12995108908598
3370000 159.585603034934
3380000 183.91899114687504
3390000 159.07634490493757
3400000 19.840325001997737
3410000 19.843922231977523
3420000 19

6140000 19.583098224293174
6150000 19.662651273951372
6160000 19.60455577399369
6170000 19.6074350742955
6180000 19.639157435352345
6190000 106.5162867100025
6200000 171.17843365878468
6210000 163.83349315011117
6220000 140.37067639790607
6230000 133.13651562025524
6240000 153.58655972305587
6250000 145.6118835390312
6260000 147.39735957911842
6270000 163.5227956149623
6280000 145.80506145344413
6290000 158.55525986007734
6300000 157.64573819321436
6310000 143.66273256347523
6320000 152.4795351495552
6330000 142.11450855294248
6340000 138.2566554170091
6350000 142.46953014139146
6360000 142.55933815993848
6370000 147.92665395288014
6380000 142.00647650896065
6390000 142.896991682562
6400000 146.58625128269807
6410000 140.86845765706988
6420000 141.80955235310097
6430000 148.0976875370393
6440000 149.23999295703717
6450000 143.8345511082857
6460000 145.4599351396415
6470000 146.67762023615012
6480000 167.96070439049578
6490000 158.24456865404767
6500000 148.80344608133268
6510000 150.92

9220000 148.57102368032864
9230000 148.0736758327303
9240000 146.51463533011076
9250000 151.7817523093595
9260000 153.40707257433073
9270000 158.58761783032648
9280000 160.69076494760088
9290000 152.43953178457033
9300000 139.4997383940992
9310000 147.15540312860054
9320000 143.74080722339252
9330000 138.8821904068665
9340000 141.62536539071053
9350000 132.2428464693276
9360000 136.55357584480984
9370000 143.71709590036915
9380000 143.28334621442795
9390000 144.70265201723703
9400000 150.07414743750545
9410000 147.51089308957734
9420000 147.1382667767176
9430000 151.90380172129645
9440000 140.55357271157666
9450000 154.8514635432977
9460000 176.98281710000194
9470000 75.04613227659077
9480000 48.53507713054589
9490000 47.23555746039816
9500000 47.806976939522364
9510000 47.96406355971994
9520000 47.181257651957594
9530000 135.85156426966617
9540000 169.41145835934904
9550000 144.4758850055409
9560000 165.53772583954418
9570000 136.35272353741965
9580000 147.358619495896
9590000 163.190

12220000 47.10581188036695
12230000 46.71745596440243
12240000 124.95411427975382
12250000 170.55539962761802
12260000 156.24266688230668
12270000 148.51686428272922
12280000 127.94028789272018
12290000 152.89689423736556
12300000 151.13977325589568
12310000 138.08134208551795
12320000 161.28082086985518
12330000 162.56636219366177
12340000 152.15987157756112
12350000 147.32891530144386
12360000 134.29473022464373
12370000 146.36172805428285
12380000 161.18581344049946
12390000 167.3711611012342
12400000 109.95859388491472
12410000 37.381296394507274
12420000 37.87104314405948
12430000 37.64964989478953
12440000 37.622581607022646
12450000 38.863618695800376
12460000 85.03799039691476
12470000 168.28819763791367
12480000 154.25609128320417
12490000 138.56890800770947
12500000 139.89912689210163
12510000 151.4635747472849
12520000 143.84398575167543
12530000 146.7305380830586
12540000 148.7426522741226
12550000 140.48877169771797
12560000 150.0841918684822
12570000 142.7226414461178
125

...

220320000 4.359981546689755
220330000 4.0079215196912426
220340000 4.052637370131075
220350000 3.693618548596349
220360000 3.3631562684543925
220370000 3.6146068202440538
220380000 3.560743586293552
220390000 3.4884822926766543
220400000 4.057586986507734
220410000 3.9105290096079837
220420000 4.311229337920043
220430000 4.795389462628603
220440000 4.995356599226613
220450000 4.012933657501658
220460000 3.5672241977630255
220470000 3.42174022461307
220480000 3.7918546526587007
220490000 3.548462609571417
220500000 3.471176414212744
220510000 3.752218110035082
220520000 3.507239048211217
220530000 4.247043703415327
220540000 4.01980942098391
220550000 4.0034949530793424
220560000 4.147686092729794
220570000 3.4223976488242944
220580000 3.5816970209276087
220590000 3.2304200710698034
220600000 3.832898809788989
220610000 3.4851705173664094
220620000 3.5793977538303996
220630000 3.4625876366547486
220640000 4.110065615727485
220650000 3.674428565158659
220660000 3.074549577356557
22067000

223200000 3.1574495503728324
223210000 2.387079380829923
223220000 2.360416545820952
223230000 2.739191803135163
223240000 2.146554024921656
223250000 3.0737763580967186
223260000 2.2314975453508232
223270000 2.793766315496829
223280000 3.0308523557829856
223290000 2.3971247539442975
223300000 2.578407104892982
223310000 2.9805467621846793
223320000 2.382486449976696
223330000 2.7176878136634035
223340000 3.368924428532978
223350000 3.0753384071455465
223360000 3.3616991682259774
223370000 2.9387402567747234
223380000 2.9382854096632665
223390000 3.0878401715337374
223400000 2.640872015717268
223410000 2.9031376454465723
223420000 2.8254511294285987
223430000 3.079051052776476
223440000 3.3393884317155216
223450000 2.289026639074293
223460000 2.341719835256537
223470000 2.366484228934937
223480000 2.391997343694071
223490000 2.6579983409795167
223500000 2.4643363642126523
223510000 2.2233450418791967
223520000 2.8544805898093584
223530000 2.245280665119436
223540000 2.5156957969011997


225940000 2.4474155126143033
225950000 2.073920956778248
225960000 1.3148018373269703
225970000 2.0422039273354278
225980000 3.4066329049579704
225990000 1.1105371909486386
226000000 1.136551186216341
226010000 3.211213494848728
226020000 2.949474288536515
226030000 2.5709965247027475
226040000 1.9041720498632988
226050000 0.5554227663421962
226060000 1.371394016307407
226070000 2.3769690461457573
226080000 2.800882211501234
226090000 2.68821181307694
226100000 1.8927209647782446
226110000 1.2732052302336758
226120000 2.110691926952766
226130000 3.15609752326862
226140000 2.0731843593025805
226150000 1.12966530279892
226160000 2.1111646573452947
226170000 2.4085563741068907
226180000 2.495709221484615
226190000 2.257134245770514
226200000 0.536102444543673
226210000 0.5407785100951527
226220000 2.2707396590695383
226230000 2.3047295139076907
226240000 2.367024405668239
226250000 1.6165743770433265
226260000 1.164726566893041
226270000 2.2443833360896046
226280000 2.83030942479819
22629

228790000 0.3629090188527306
228800000 0.4638473446839452
228810000 0.694911846671164
228820000 1.8467208563749065
228830000 1.6376403007610243
228840000 1.2011429871696897
228850000 1.613267062837581
228860000 1.8331571189262867
228870000 1.622208554909037
228880000 1.206193874191887
228890000 1.09226301563257
228900000 1.2160134785153072
228910000 1.032478641337673
228920000 0.7212367568130691
228930000 0.5527049303896693
228940000 0.3225668840059373
228950000 0.41927858754299085
228960000 0.6080285958041616
228970000 0.955428565987673
228980000 1.2807227975034714
228990000 1.1198810081662338
229000000 1.1950607120706636
229010000 1.6366942026951512
229020000 1.6028645314618217
229030000 1.4946373106435538
229040000 1.3765714094819426
229050000 1.4828423997715514
229060000 0.7955329816432396
229070000 0.6241112028195858
229080000 1.0662042141900723
229090000 0.2211958119080199
229100000 0.8092062949969372
229110000 0.7362029386146067
229120000 0.610718854939123
229130000 1.2926028727

231460000 0.46979753933467466
231470000 0.5294999680065314
231480000 0.24799185402863555
231490000 0.521005249836968
231500000 0.40291693682664637
231510000 0.5294810652831263
231520000 0.41258176686441894
231530000 0.2906952506170074
231540000 0.4992594101431105
231550000 0.48446331222497757
231560000 0.7177701382254164
231570000 0.32900455083472196
231580000 0.27795338075601395
231590000 0.4840919560337663
231600000 0.5126219814209475
231610000 0.6847740323877335
231620000 0.3545223134560585
231630000 0.3202489201932152
231640000 0.40249678251741994
231650000 0.4967705262316664
231660000 0.6589169430642392
231670000 0.3418261887590223
231680000 0.29404469835448266
231690000 0.4305503132323292
231700000 0.4369603821703593
231710000 0.629185663525184
231720000 0.36394030325129956
231730000 0.24312763618506325
231740000 0.3313687426289717
231750000 0.4671446128774418
231760000 0.47545153637233045
231770000 0.4063592817898989
231780000 0.25471936848291427
231790000 0.24746118955487678
23